## Importações:

In [1]:
import numpy as np
import random as rd
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import matplotlib.animation as animation
from itertools import combinations
from scipy.optimize import curve_fit
#from scipy.interpolate import interp1d
plt.style.use('seaborn-bright')
import pandas as pd

In [2]:
from funcoes import Particula, mudar_passo, gerar_particulas, simular_reacao

## Parâmetros da simulação:

In [ ]:
#Parâmetros da simulação:

numero_particulas = 100
tamanho_caixa = 50
RAIO = 0.5
MASSA = 1
probabilidade_reacao = 0.7
tipo = 'atomo'

tempo_final = 10
numero_passos = 700
tempo_cada_passo = tempo_final/numero_passos

#lista_particulas = gerar_particulas(numero_particulas, RAIO, MASSA, tamanho_caixa, tipo)

## Simulação da reação química:

- Se ocorre uma reação ela é necessariamente 
- Tem conservação de massa?
- Eu preciso que em algum lugar seja definido o tipo das partículas para eu conseguir fazer um if 
- talvez transformar nossa partícula em uma matriz igual do amauri e tirar o resto das classes
- precisa atualizar a matriz de partículas 
- verificar se há colisão
- Testar se é melhor fazer uma nova classe 'molécula' ou usar 'nova_particula = Particula(nova_massa, novo_raio, nova_posicao, nova_velocidade, tipo)
- Checar se a conservação de volume está no livro
- Tirar uma das partículas e gerar uma nova a partir da outra
- Usar o itertools (tem uma função que não repete) para usar o particula1 com particula2
- itertools.combinations

In [ ]:
lista_particulas = gerar_particulas(numero_particulas, RAIO, MASSA, tamanho_caixa, tipo)

lista_atomos, lista_moleculas = [], []

# Compute simulation (It takes some time if numero_passos and numero_particulas are large)
for i in range(numero_passos):
     # Simular colisões
    simular_reacao(lista_particulas, probabilidade_reacao)

    # Atualizar posições e velocidades
    mudar_passo(lista_particulas, tempo_cada_passo, tamanho_caixa)
    
    a_count, m_count = 0, 0
    
    for particula in lista_particulas:
        if particula.tipo == 'atomo':
            a_count += 1
        else:
            m_count += 1
    
    lista_atomos.append(a_count)
    lista_moleculas.append(m_count)

plt.scatter(list(range(numero_passos)),lista_atomos,label='átomos')
plt.scatter(list(range(numero_passos)),lista_moleculas,label='moléculas')
plt.legend()

## Gerar animação da reação:

In [ ]:
lista_atomos2, lista_moleculas2 = [], []

def criar_animacao_simulacao_reacao(lista_particulas, tamanho_caixa, RAIO, NUM_STEPS, probabilidade_reacao):
    fig, ax = plt.subplots(figsize=(10, 10))
    ims = []

    ax.set_xlim(0, tamanho_caixa)
    ax.set_ylim(0, tamanho_caixa)
    
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title('Simulação de Reação Química', fontsize=20)
     
    for k in range(NUM_STEPS):
        # Chamando a função para simular reações
        simular_reacao(lista_particulas, probabilidade_reacao)

        # Desenha as partículas após a simulação de reação
        circles = []
        a_count, m_count = 0, 0
        
        for particula in lista_particulas:
            if k < len(particula.todas_posicoes):
                x = particula.todas_posicoes[k][0]
                y = particula.todas_posicoes[k][1]
                circle = plt.Circle((x, y), RAIO, fill=True, color='blue', edgecolor='black')
                ax.add_artist(circle)
                circles.append(circle)
            
            if particula.tipo == 'atomo':
                a_count += 1
            else:
                m_count += 1
        
        lista_atomos2.append(a_count)
        lista_moleculas2.append(m_count)
        
        ims.append(circles)

    ani = animation.ArtistAnimation(fig, ims, interval=40, blit=True, repeat_delay=100)
    writer = animation.PillowWriter(fps=10, bitrate=4000)
    ani.save("resultados/simulacao_reacao.gif", writer=writer, dpi=100)

criar_animacao_simulacao_reacao(lista_particulas, tamanho_caixa, RAIO, numero_passos, probabilidade_reacao)

C:\Users\DANIEL~1\AppData\Local\Temp/ipykernel_9972/751259133.py:26: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  circle = plt.Circle((x, y), RAIO, fill=True, color='blue', edgecolor='black')


## Criando DataFrame para trabalhar com os dados:

In [ ]:
lista = {'Atomos': lista_atomos, 'Moléculas': lista_moleculas, 'tempo': range(numero_passos)}

print(len(lista_atomos))
print(len(lista_moleculas))
print(len(range(numero_passos)))


df = pd.DataFrame(lista)

atomodf = df['Atomos'] #df.iloc[0 : 70, 0]
temporeagente = df['tempo'] #df.iloc[0:70, 2]

produtodf = df.iloc[1 : 99, 1]
tempoproduto = df.iloc[1 : 99, 2]



In [ ]:
display(df)

In [ ]:
df.to_csv('dados.csv', index=False)

## Fitagem com função exponencial:

In [ ]:
def exponencial(t, a, k, c):
    return a * np.exp(-k * t) + c

popt, _ = curve_fit(exponencial, temporeagente, atomodf)
print(popt)

valores_fit_reagente = exponencial(temporeagente, *popt)

k = popt[1]
print("O valor de k é:", k)

plt.scatter(temporeagente, atomodf , label='Dados Originais')
plt.plot(temporeagente, valores_fit_reagente, label='Ajuste Exponencial', color='red')


plt.legend()
plt.show()

## Fitagem com função logarítmica:

In [ ]:
def logaritimica(x, a, b, c):
    return a * np.log(b * x) + c

popt, _ = curve_fit(logaritimica, tempoproduto, produtodf)
print(popt)
valores_fit_produto = logaritimica(tempoproduto, *popt)

b = popt[1]
print("O valor de k é:", b)

plt.scatter(temporeagente, atomodf , label='Dados Originais')
plt.plot(temporeagente, valores_fit_reagente, label='Ajuste Exponencial', color='red')
plt.scatter(tempoproduto, produtodf , label='Dados Originais', color='green')
plt.plot(tempoproduto, valores_fit_produto, label='Ajuste Exponencial', color='purple')
plt.legend()
plt.show()

## Análise do Método da Velocidade Inicial:

In [ ]:
# Vlor de K pela quantidade de partículas iniciais
# Initial rate method

In [ ]:
numero_particulas = [20, 60] #100, 140, 180, 200]

parametros_ajustados = []
valores_ajustados = []

for num_particulas in numero_particulas:
    lista_particulas = gerar_particulas(num_particulas, RAIO, MASSA, tamanho_caixa, tipo)
    lista_atomos =  []

    # Compute simulation
    for i in range(numero_passos):
        # Simular colisões
        simular_reacao(lista_particulas, probabilidade_reacao)

        # Atualizar posições e velocidades
        mudar_passo(lista_particulas, tempo_cada_passo, tamanho_caixa)

        a_count, m_count = 0, 0

        for particula in lista_particulas:
            if particula.tipo == 'atomo':
                a_count += 1
            else:

        lista_atomos.append(a_count)
       
    temporeagente = range(numero_passos)  # Certifique-se de ter o tempo correto aqui
    atomodf = lista_atomos  # Certifique-se de ter os dados corretos aqui

    popt, _ = curve_fit(exponencial, temporeagente, atomodf)
    parametros_ajustados.append(popt)
    valores_fit_reagente = exponencial(temporeagente, *popt)
    valores_ajustados.append(valores_fit_reagente)

# Agora, você pode usar os resultados para fazer o que deseja
# Por exemplo, você pode imprimir os parâmetros ajustados
for i, popt in enumerate(parametros_ajustados):
    print(f"Para N = {numero_particulas[i]}, os parâmetros ajustados são: a={popt[0]}, k={popt[1]}, c={popt[2]}")

# Ou plotar os resultados
plt.figure(figsize=(10, 5))
for i, num_particulas in enumerate(numero_particulas):
    plt.plot(temporeagente, valores_ajustados[i], label=f'N = {num_particulas}')

    
plt.legend()
plt.show()


#plt.scatter(range(numero_passos), lista_atomos, label= f'N = {num_particulas}')
#plt.legend()
#plt.show()  # Adicionei um comando para mostrar o gráfico a cada iteração (remova se desejar plotar todas juntas)


In [ ]:
#plotar K pela concentração

----

---

---

---

In [ ]:
# Calcular a diferença no tempo (assumindo que o intervalo de tempo é constante)
delta_t_novo = df['tempo'][1] - df['tempo'][0]

# Calcular a derivada da concentração ajustada
velocidade_reacao_ajustada = np.gradient(valores_fit_produto, delta_t_novo)

# taxa de variação da concentração de átomos ajustada em função do tempo
# A velocidade de reação ajustada é d[atomos]/dtempo

# plotando a velocidade de reação ajustada
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(df['tempo'][:-1], velocidade_reacao_ajustada, '-', label='Velocidade de Reação Ajustada')
plt.legend()
plt.title('Velocidade de Reação Ajustada')
plt.xlabel('Tempo')
plt.ylabel('Velocidade de Reação')
plt.show()

In [ ]:
import numpy as np
from lmfit import Model
import matplotlib.pyplot as plt

# 1. Defina a função exponencial
def exponencial(t, a, k, c):
    return a * np.exp(-k * t) + c

# 2. Crie um Modelo com o lmfit
modelo = Model(exponencial)

# 3. Crie Parâmetros Iniciais
params = modelo.make_params(a=1, k=1, c=1)

# 4. Dados de Exemplo (Substitua com seus próprios dados)
tempo = np.array([0, 1, 2, 3, 4, 5])
valores = np.array([10, 7, 5, 3, 2, 1])

# 5. Realize o Ajuste
resultados = modelo.fit(atomodf, t=df['tempo'], params=params)

# 6. Obtenha os Resultados do Ajuste
print(resultados.fit_report())

# 7. Visualize o Ajuste
plt.scatter(tempo, valores, label='Dados Originais')
plt.plot(tempo, resultados.best_fit, label='Ajuste Exponencial', color='red')
plt.legend()
plt.show()


## Fitagem com Spline Cúbica:

In [ ]:
# Tempo e concentração de átomos
tempo = np.arange(numero_passos)

# Interpolação utilizando spline cúbica
fa = interp1d(tempo, lista_atomos, kind='cubic')
fm = interp1d(tempo, lista_moleculas, kind='cubic')

# Novos pontos de tempo para a curva ajustada 
tempo_novo = np.linspace(tempo.min(), tempo.max(), 10000)

# Avaliação da função ajustada
concentracao_ajustada_atomos = fa(tempo_novo)
concentracao_ajustada_moleculas = fm(tempo_novo)


# Agora você tem a concentração ajustada em função do tempo

# plotar os dados e a curva ajustada
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(tempo, lista_atomos, 'o', label='Átomos inicial ')
plt.plot(tempo_novo, concentracao_ajustada_atomos, '-', label='Átomos com ajuste com Spline Cúbica')

plt.plot(tempo, lista_moleculas, 'o', label='Móleculas inicial ')
plt.plot(tempo_novo, concentracao_ajustada_moleculas, '-', label='Móleculas com ajuste com Spline Cúbica')
plt.legend()
plt.title('Ajuste com Spline Cúbica')
plt.xlabel('Tempo')
plt.ylabel('Concentração dos reagentes e produtos')
plt.show()


## Gráfico da derivada da concentração por tempo: